# Benchmark of various outlier detection models

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). The full result by averaging 10 indepent trials can be found [here](https://pyod.readthedocs.io/en/latest/benchmark.html).

**[PyOD](https://github.com/yzhao062/pyod)** is a comprehensive **Python toolkit** to **identify outlying objects** in 
multivariate data with both unsupervised and supervised approaches.
The model covered in this example includes:

  1. Linear Models for Outlier Detection:
     1. **PCA: Principal Component Analysis** use the sum of
       weighted projected distances to the eigenvector hyperplane 
       as the outlier outlier scores)
     2. **MCD: Minimum Covariance Determinant** (use the mahalanobis distances 
       as the outlier scores)
     3. **OCSVM: One-Class Support Vector Machines**
     
  2. Proximity-Based Outlier Detection Models:
     1. **LOF: Local Outlier Factor**
     2. **CBLOF: Clustering-Based Local Outlier Factor**
     3. **kNN: k Nearest Neighbors** (use the distance to the kth nearest 
     neighbor as the outlier score)
     4. **HBOS: Histogram-based Outlier Score**
     
  3. Probabilistic Models for Outlier Detection:
     1. **ABOD: Angle-Based Outlier Detection**
  
  4. Outlier Ensembles and Combination Frameworks
     1. **Isolation Forest**
     2. **Feature Bagging**

     
Corresponding file could be found at /examples/compare_all_models.py

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [2]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']
roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    classifiers = {'Angle-based Outlier Detector (ABOD)': ABOD(
        contamination=outliers_fraction),
        'Cluster-based Local Outlier Factor': CBLOF(
            contamination=outliers_fraction, check_estimator=False,
            random_state=random_state),
        'Feature Bagging': FeatureBagging(contamination=outliers_fraction,
                                          check_estimator=False,
                                          random_state=random_state),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=outliers_fraction),
        'Isolation Forest': IForest(contamination=outliers_fraction,
                                    random_state=random_state),
        'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
        'Local Outlier Factor (LOF)': LOF(
            contamination=outliers_fraction),
        'Minimum Covariance Determinant (MCD)': MCD(
            contamination=outliers_fraction, random_state=random_state),
        'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction,
                                       random_state=random_state),
        'Principal Component Analysis (PCA)': PCA(
            contamination=outliers_fraction, random_state=random_state),
    }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{clf_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
            clf_name=clf_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571, execution time: 2.0324s
Cluster-based Local Outlier Factor ROC:0.7824, precision @ rank n:0.4643, execution time: 0.8663s
Feature Bagging ROC:0.7796, precision @ rank n:0.4643, execution time: 0.5355s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714, execution time: 0.8483s
Isolation Forest ROC:0.8639, precision @ rank n:0.6071, execution time: 0.2376s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5, execution time: 0.0902s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643, execution time: 0.0662s
Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286, execution time: 1.1842s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5, execution time: 0.0471s
Principal Component Analysis (PCA) ROC:0.7997, precision @ rank n:0.5, execution time: 0.0521s

... Processing cardio.mat ...
Angle-based Outlier Det

One-class SVM (OCSVM) ROC:0.5171, precision @ rank n:0.0, execution time: 1.3646s
Principal Component Analysis (PCA) ROC:0.526, precision @ rank n:0.0, execution time: 0.0411s

... Processing pendigits.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.6693, precision @ rank n:0.0635, execution time: 1.3035s
Cluster-based Local Outlier Factor ROC:0.9207, precision @ rank n:0.1429, execution time: 0.1775s
Feature Bagging ROC:0.449, precision @ rank n:0.0635, execution time: 3.4542s
Histogram-base Outlier Detection (HBOS) ROC:0.9311, precision @ rank n:0.2381, execution time: 0.008s
Isolation Forest ROC:0.9547, precision @ rank n:0.3651, execution time: 0.5314s
K Nearest Neighbors (KNN) ROC:0.7417, precision @ rank n:0.0794, execution time: 0.7089s
Local Outlier Factor (LOF) ROC:0.451, precision @ rank n:0.0794, execution time: 0.5354s
Minimum Covariance Determinant (MCD) ROC:0.8182, precision @ rank n:0.0794, execution time: 2.0334s
One-class SVM (OCSVM) ROC:0.9291, precision @ rank n:0.

In [3]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,2.0324,0.8663,0.5355,0.8483,0.2376,0.0902,0.0662,1.1842,0.0471,0.0521
0,cardio,1831,21,9.6122,0.369,0.1023,0.753,0.007,0.2346,0.1765,0.0962,0.5244,0.0742,0.003
0,glass,214,9,4.2056,0.0411,0.0391,0.0381,0.002,0.1795,0.012,0.003,0.0381,0.002,0.001
0,ionosphere,351,33,35.8974,0.0732,0.0371,0.0692,0.006,0.1595,0.021,0.005,0.0712,0.005,0.002
0,letter,1600,32,6.25,0.3078,0.0892,0.6708,0.007,0.2456,0.1544,0.0872,1.0648,0.0782,0.004
0,lympho,148,18,4.0541,0.0281,0.0351,0.0271,0.003,0.1604,0.009,0.002,0.025,0.001,0
0,mnist,7603,100,9.2069,6.5705,0.8824,43.9129,0.0411,1.6304,6.1113,5.9909,4.3596,4.4559,0.1303
0,musk,3062,166,3.1679,1.9422,0.2778,12.1263,0.0491,0.9355,1.6624,1.5612,20.0002,1.1771,0.1413
0,optdigits,5216,64,2.8758,2.1958,0.37,11.2941,0.0241,0.6447,1.7828,1.488,1.532,1.3646,0.0411
0,pendigits,6870,16,2.2707,1.3035,0.1775,3.4542,0.008,0.5314,0.7089,0.5354,2.0334,0.8583,0.007


Analyze the performance of ROC and Precision @ n

In [4]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7824,0.7796,0.8511,0.8639,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5718,0.971,0.6362,0.8297,0.9172,0.7327,0.5822,0.8134,0.9414,0.9571
0,glass,214,9,4.2056,0.7792,0.8438,0.825,0.7438,0.7312,0.8229,0.8438,0.7771,0.5688,0.6125
0,ionosphere,351,33,35.8974,0.9146,0.783,0.9246,0.6094,0.8416,0.9307,0.9179,0.968,0.8235,0.7871
0,letter,1600,32,6.25,0.8799,0.5095,0.8742,0.589,0.6528,0.8718,0.8744,0.7681,0.6273,0.5487
0,lympho,148,18,4.0541,1,1,1,1,1,1,1,1,1,1
0,mnist,7603,100,9.2069,0.7773,0.7853,0.7215,0.5674,0.7863,0.8451,0.7194,0.8786,0.8474,0.8492
0,musk,3062,166,3.1679,0.2161,0.9899,0.473,0.9999,1,0.8009,0.4629,1,1,1
0,optdigits,5216,64,2.8758,0.4894,0.5329,0.5062,0.8774,0.6735,0.406,0.5277,0.3822,0.5171,0.526
0,pendigits,6870,16,2.2707,0.6693,0.9207,0.449,0.9311,0.9547,0.7417,0.451,0.8182,0.9291,0.9339


In [5]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.4643,0.5714,0.6071,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.2667,0.7867,0.2533,0.44,0.4533,0.36,0.2267,0.4,0.5333,0.68
0,glass,214,9,4.2056,0.1667,0.1667,0.1667,0.1667,0.1667,0.1667,0.1667,0,0.1667,0.1667
0,ionosphere,351,33,35.8974,0.8235,0.549,0.7843,0.4118,0.6078,0.8627,0.7647,0.8627,0.6667,0.5686
0,letter,1600,32,6.25,0.375,0.0938,0.3438,0.0625,0.0938,0.25,0.3125,0.0938,0.0625,0.0625
0,lympho,148,18,4.0541,1,1,1,1,1,1,1,1,1,1
0,mnist,7603,100,9.2069,0.3733,0.3322,0.3185,0.1096,0.2842,0.4178,0.3322,0.4589,0.3904,0.387
0,musk,3062,166,3.1679,0.1,0.65,0.125,0.975,1,0.175,0.125,1,1,1
0,optdigits,5216,64,2.8758,0.0152,0,0.0303,0.2121,0.0303,0,0.0303,0,0,0
0,pendigits,6870,16,2.2707,0.0635,0.1429,0.0635,0.2381,0.3651,0.0794,0.0794,0.0794,0.3175,0.3175
